In [1]:
from pyspark.sql import SparkSession
from delta import *
import pandas as pd
import numpy as np

builder = SparkSession.builder.appName("flight-fusion-test") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

22/06/02 00:00:33 WARN Utils: Your hostname, DESKTOP-ME6BQMM resolves to a loopback address: 127.0.1.1; using 192.168.206.34 instead (on interface eth0)
22/06/02 00:00:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/robstar/.cache/pypoetry/virtualenvs/workspace-flight-fusion-jOezzoJ6-py3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/robstar/.ivy2/cache
The jars for the packages stored in: /home/robstar/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f49eb3de-51f5-45b1-9811-014a2ed9663c;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.2.1 in central
	found io.delta#delta-storage;1.2.1 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/1.2.1/delta-core_2.12-1.2.1.jar ...
	[SUCCESSFUL ] io.delta#delta-core_2.12;1.2.1!delta-core_2.12.jar (232ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/1.2.1/delta-storage-1.2.1.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;1.2.1!delta-storage.jar (20ms)
:: resolution report :: resolve 1386ms :: artifacts dl 258ms
	:: modules in use:
	io.delta#delta-core_2.12;1.2.1 from central in [default]
	io.delta#delta-storag

In [12]:
import os
from pathlib import Path
from subprocess import STDOUT, check_output

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pyarrow.fs import LocalFileSystem

DATA_ROOT = "_ff_data"

def file_relative_path(dunderfile, relative_path):
    return os.path.join(os.path.dirname(dunderfile), relative_path)


def workspace_root() -> Path:
    output = check_output(["cargo", "metadata"], stderr=STDOUT).decode()  # nosec
    key = 'workspace_root":"'
    idx = output.find(key)
    part = output[idx + len(key) :]
    idx = part.find('"')

    return Path(part[:idx])

data_path = workspace_root() / "test" / "db"

In [41]:
ts = pd.date_range(
    start=pd.to_datetime("2020-01-01 00:00:00"),
    end=pd.to_datetime("2020-01-04 00:00:00"),
    freq="3h",
)

table = pa.Table.from_pydict(
    {
        "timestamp": ts,
        "date":  pd.to_datetime(pd.Series(ts)).dt.date,
        "string": np.random.choice(["a", "b", "c"], len(ts)),
        "double": np.random.randn(len(ts)),
        "real": np.random.randn(len(ts)),
        "float": np.random.randn(len(ts)),
    }
)
table.schema, table.shape


(timestamp: timestamp[ns]
 date: date32[day]
 string: string
 double: double
 real: double
 float: double,
 (25, 6))

In [42]:
sdf = spark.createDataFrame(table.to_pandas()).repartition(1)

sdf.write.format("delta").save(str(data_path / "delta" / DATA_ROOT / "simple"), mode="overwrite")
sdf.write.format("delta").partitionBy("date").save(
    str(data_path / "delta" / "partitioned" / DATA_ROOT / "date"), mode="overwrite"
)
sdf.write.format("delta").partitionBy("string").save(
    str(data_path / "delta" / "partitioned" / DATA_ROOT / "string"), mode="overwrite"
)


In [23]:
sdf.write.format("delta").partitionBy("part").save(str(data_path / DATA_ROOT / "rand"), mode="overwrite")

In [13]:
DeltaTable.createIfNotExists(spark) \
  .tableName("people10m") \
  .addColumn("id", "INT") \
  .addColumn("firstName", "STRING") \
  .addColumn("middleName", "STRING") \
  .addColumn("lastName", "STRING", comment = "surname") \
  .addColumn("gender", "STRING") \
  .addColumn("birthDate", "TIMESTAMP") \
  .addColumn("ssn", "STRING") \
  .addColumn("salary", "INT") \
  .location("tmp/people") \
  .execute()

AnalysisException: Database 'delta' not found